In [ ]:
import os
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from functools import partial
import optuna
import gc
from typing import Literal
import torch.nn.functional as F
import math

# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN


# Set the random seed
set_seed()


In [ ]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [3]:
def train(
    data_loader,
    model,
    optimizer,
    criterion,
    device,
    save_checkpoints,
    checkpoint_path,
    current_epoch,
):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader), correct / total

In [4]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return total_loss / len(data_loader), accuracy
    return predictions

In [5]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({"id": test_graph_ids, "pred": predictions})

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [6]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color="blue")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss per Epoch")

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color="green")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training Accuracy per Epoch")

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [7]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (
            1
            - torch.nn.functional.one_hot(targets, num_classes=logits.size(1))
            .float()
            .sum(dim=1)
        )
        return (losses * weights).mean()

In [ ]:
class SCELoss(torch.nn.Module):
    def __init__(self, num_classes: int = 6, alpha: float = 0.1, beta: float = 1.0):
        super().__init__()
        self.alpha, self.beta = alpha, beta
        self.num_classes = num_classes

    def forward(self, logits, targets):
        # CCE
        ce = F.cross_entropy(logits, targets, reduction="none")

        # RCE
        pred = F.softmax(logits, dim=1).clamp(min=1e-6, max=1 - 1e-6)
        one_hot = F.one_hot(targets, self.num_classes).float()
        rce = -(1 - one_hot) * torch.log(1 - pred)
        rce = rce.sum(dim=1)
        return (self.alpha * ce + self.beta * rce).mean()

In [ ]:
class NCODLoss(torch.nn.Module):
    def __init__(
        self,
        dataset,
        embedding_dimensions: int = 300,
        total_epochs: int = 150,
        *args,
        **kwargs,
    ) -> None:
        super().__init__(*args, **kwargs)
        label_counts = {}
        for elem in dataset:
            y = elem.y.item()
            label_counts[y] = label_counts.get(y, 0) + 1

        self.num_elements = len(dataset)
        self.num_classes = len(label_counts)
        self.embedding_dimensions = embedding_dimensions
        self.total_epochs = total_epochs

        # u_i: Learnable noise discount scalar per sample
        # This parameter is updated during training to reflect the estimated noisiness of each sample.
        # If u_i grows, the loss becomes less sensitive to this sample.
        # Initialized near zero (assumes most samples are initially clean).
        self.u = nn.Parameter(torch.empty(self.num_elements, 1))
        torch.nn.init.normal_(self.u, mean=1e-8, std=1e-9)

        # Buffer for past feature embeddings, used for centroid computation
        # Shape: [num_samples, embedding_dimensions]
        # These are updated each time a sample appears in a batch
        self.past_embeddings = torch.rand((self.num_elements, embedding_dimensions))

        # Per-class centroids. used to compute soft labels
        # Initialized randomly and updated each epoch using confident samples
        self.centroids = torch.rand((self.num_classes, embedding_dimensions))

        # Bins: stores indices for each class neded for centroid computation
        # self.bins[i] will contain the list of dataset indices where label == i
        self.bins = torch.tensor(sorted(list(label_counts.items())), dtype = torch.uint32)
        # ratio_consistency: strength of consistency regularization between model outputs
        # Helps enforce prediction stability across augmentations or dropout
        # self.ratio_consistency = ratio_consistency

        # ratio_balance: strength of class balance regularization (KL divergence to uniform prior)
        # Helps prevent the model from collapsing to over-predicting certain classes under label noise
        # Tune if class imbalance becomes a problem or you're using soft labels heavily.
        # self.ratio_balance = ratio_balance

    def forward(
        self,
        logits: torch.Tensor,
        indexes: torch.Tensor,
        embeddings: torch.Tensor,
        epoch: int,
    ):
        embeddings = F.normalize(embeddings, dim=1)
        if epoch ==0:
            percent = math.ceil((50 - (50 / self.total_epochs) * epoch) + 50)
            u_class= self.u.numpy()[indexes.numpy()]
            bottom_k = int((len(u_class) / 100) * percent)

SyntaxError: expected ':' (3521734483.py, line 56)

In [ ]:
def objective(
    trial,
    train_loader,
    val_loader,
    num_checkpoints,
    checkpoints_dir,
    run_name,
    best_model_path,
    logs_dir,
    *,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):  # -> float | Any:
    logging.info("#" * 80)
    # Hyperparameter search space
    logging.info("Start case study with parameters:")
    gnn_type = trial.suggest_categorical(
        "gnn_type", ["gin", "gin-virtual", "gcn", "gcn-virtual"]
    )
    drop_ratio = trial.suggest_float("dropout", 0.0, 0.7)
    num_layers = trial.suggest_int("num_layers", 3, 6)
    embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 300, 600])
    num_epochs = trial.suggest_int("num_epochs", 40, 80, step=20)

    logging.info(f"{gnn_type=}")
    logging.info(f"{drop_ratio=}")
    logging.info(f"{num_layers=}")
    logging.info(f"{embedding_dim=}")
    logging.info(f"{num_epochs=}")

    # Initialize model
    model = GNN(
        gnn_type="gin" if "gin" in gnn_type else "gcn",
        num_class=6,
        num_layer=num_layers,
        emb_dim=embedding_dim,
        drop_ratio=drop_ratio,
        virtual_node="virtual" in gnn_type,
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters())
    criterion = SCELoss()  # torch.nn.CrossEntropyLoss()

    # Prepare checkpoints
    checkpoint_epochs = [
        int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)
    ]

    best_val_accuracy = 0.0
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader,
            model,
            optimizer,
            criterion,
            device,
            save_checkpoints=(epoch + 1 in checkpoint_epochs),
            checkpoint_path=os.path.join(checkpoints_dir, f"model_{run_name}"),
            current_epoch=epoch,
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        msg = (
            f"[{run_name}] Epoch {epoch + 1}/{num_epochs} | "
            f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}"
        )
        logging.info(msg)

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), best_model_path)
            logging.info(f"[{run_name}] Best model updated at {best_model_path}")

    plot_training_progress(
        train_losses, train_accuracies, os.path.join(logs_dir, "train_plots")
    )
    plot_training_progress(
        val_losses, val_accuracies, os.path.join(logs_dir, "val_plots")
    )
    logging.info(f"Case study end, {best_val_accuracy}")
    logging.info("#" * 80)
    logging.info("\n")

    return best_val_accuracy

In [ ]:
def case_study(
    dataset_name: Literal["A", "B", "C", "D"],
    n_trials: int = 30,
    resume_if_exists: bool = True,
    num_checkpoints: int = 10,
    default_batch_size: int = 32,
):
    script_root = os.getcwd()
    train_path = f"./datasets/{dataset_name}/train.json.gz"
    run_name = dataset_name

    logs_dir = os.path.join(script_root, "logs", run_name)
    os.makedirs(logs_dir, exist_ok=True)
    logging.basicConfig(
        filename=os.path.join(logs_dir, "training.log"),
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        filemode="w",
    )
    logging.getLogger().addHandler(logging.StreamHandler())

    checkpoints_dir = os.path.join(script_root, "checkpoints", run_name)
    best_model_path = os.path.join(checkpoints_dir, f"model_{run_name}_best.pth")
    summary_csv_path = os.path.join(logs_dir, f"optuna_summary_{dataset_name}.csv")
    os.makedirs(checkpoints_dir, exist_ok=True)

    full_dataset = GraphDataset(train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(
        full_dataset, [train_size, val_size], generator=generator
    )

    train_loader = DataLoader(
        train_dataset,  # type:ignore
        batch_size=default_batch_size,
        shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset,  # type:ignore
        batch_size=default_batch_size,
        shuffle=False,
    )

    logging.info(f"--- Starting Optuna optimization for dataset {dataset_name} ---")

    study = optuna.create_study(study_name=run_name, direction="maximize")

    obj = partial(
        objective,
        train_loader=train_loader,
        val_loader=val_loader,
        num_checkpoints=num_checkpoints,
        checkpoints_dir=checkpoints_dir,
        run_name=run_name,
        best_model_path=best_model_path,
        logs_dir=logs_dir,
    )
    study.optimize(obj, n_trials=n_trials)

    all_trials = []
    for trial in study.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            row = {"accuracy": trial.value}
            row.update(trial.params)
            all_trials.append(row)

    results_df = pd.DataFrame(all_trials)
    results_df.to_csv(summary_csv_path, index=False)

    logging.info(f"\nAll trials saved to: {summary_csv_path}")
    logging.info(f"\nBest result for dataset {dataset_name}:")
    display(results_df.sort_values("accuracy", ascending=False))
    logging.info(f"\nBest Params for {dataset_name}:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")

    del train_loader, val_loader, full_dataset, train_dataset, val_dataset
    gc.collect()


In [11]:
case_study("A", 2)

--- Starting Optuna optimization for dataset A ---
[I 2025-05-25 19:03:44,730] A new study created in memory with name: A
################################################################################
Start case study with parameters:
gnn_type='gcn'
drop_ratio=0.25287334651446963
num_layers=6
embedding_dim=64
num_epochs=60
Epoch:   0%|          | 0/60 [00:00<?, ?it/s][A] Epoch 1/60 | Train Loss: 1.0281, Acc: 0.3729 | Val Loss: 1.5257, Acc: 0.4034
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:   2%|▏         | 1/60 [00:28<28:05, 28.57s/it][A] Epoch 2/60 | Train Loss: 0.9751, Acc: 0.4359 | Val Loss: 1.5691, Acc: 0.4047
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:   3%|▎         | 2/60 [00:52<25:05, 25.97s/it][A] Epoch 3/60 | Train Loss: 0.9505, Acc: 0.4662 | Val Loss: 1.3864, Acc: 0.5084
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackato

Checkpoint saved at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_6.pth


[A] Epoch 6/60 | Train Loss: 0.8982, Acc: 0.5244 | Val Loss: 1.3568, Acc: 0.5204
Epoch:  10%|█         | 6/60 [02:28<21:50, 24.28s/it][A] Epoch 7/60 | Train Loss: 0.8879, Acc: 0.5356 | Val Loss: 1.3109, Acc: 0.5461
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  13%|█▎        | 8/60 [03:16<20:47, 24.00s/it][A] Epoch 9/60 | Train Loss: 0.8634, Acc: 0.5622 | Val Loss: 1.2893, Acc: 0.5598
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  15%|█▌        | 9/60 [03:40<20:18, 23.90s/it][A] Epoch 10/60 | Train Loss: 0.8571, Acc: 0.5725 | Val Loss: 1.2592, Acc: 0.5833
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  18%|█▊        | 11/60 [04:27<19:27, 23.82s/it]

Checkpoint saved at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_12.pth


[A] Epoch 12/60 | Train Loss: 0.8378, Acc: 0.5944 | Val Loss: 1.2404, Acc: 0.5949
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  22%|██▏       | 13/60 [05:15<18:36, 23.76s/it][A] Epoch 14/60 | Train Loss: 0.8212, Acc: 0.6082 | Val Loss: 1.2530, Acc: 0.5971
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  23%|██▎       | 14/60 [05:39<18:15, 23.81s/it][A] Epoch 15/60 | Train Loss: 0.8158, Acc: 0.6103 | Val Loss: 1.2216, Acc: 0.6157
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  25%|██▌       | 15/60 [06:03<17:55, 23.90s/it][A] Epoch 16/60 | Train Loss: 0.8094, Acc: 0.6148 | Val Loss: 1.1910, Acc: 0.6268
[A] Best model updated at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_best.pth
Epoch:  28%|██▊       | 17/60 [06:56<17:56, 25.03s/it]

...

Checkpoint saved at /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_80.pth


[A] Epoch 80/80 | Train Loss: 0.8088, Acc: 0.6268 | Val Loss: 1.3899, Acc: 0.5288
Epoch: 100%|██████████| 80/80 [32:52<00:00, 24.66s/it]
Case study end, 0.6050531914893617
################################################################################


[I 2025-05-25 20:00:53,617] Trial 1 finished with value: 0.6050531914893617 and parameters: {'gnn_type': 'gcn-virtual', 'dropout': 0.5780210233709472, 'num_layers': 3, 'embedding_dim': 128, 'num_epochs': 80}. Best is trial 0 with value: 0.661790780141844.

All trials saved to: /home/haislich/Documents/noisy_labels/hackaton/logs/A/optuna_summary_A.csv

Best result for dataset A:


,accuracy,gnn_type,dropout,num_layers,embedding_dim,num_epochs
0,0.661791,gcn,0.252873,6,64,60
1,0.605053,gcn-virtual,0.578021,3,128,80



Best Params for A:


  gnn_type: gcn
  dropout: 0.25287334651446963
  num_layers: 6
  embedding_dim: 64
  num_epochs: 60
